In [3]:
import pandas as pd
cars = pd.read_csv('cars_cleaned.csv')
cars.columns

Index(['condition', 'drive', 'fuel', 'location', 'odometer', 'paint color',
       'price', 'sale type', 'title status', 'transmission', 'type',
       'cylinders', 'size', 'latitude', 'longitude', 'year', 'age', 'make_',
       'model', 'pos_words', 'neg_words', 'low_text'],
      dtype='object')

In [21]:
cars.shape

(37341, 22)

In [6]:
cars.head()

,condition,drive,fuel,location,odometer,paint color,price,sale type,title status,transmission,...,size,latitude,longitude,year,age,make_,model,pos_words,neg_words,low_text
0,good,4wd,gas,vancouver,281000,white,5500,owner,rebuilt,automatic,...,unknown,49.234833,-123.090560,2007,14,bmw,x3,False,False,False
1,unknown,unknown,gas,vancouver,140000,unknown,600,owner,clean,automatic,...,unknown,49.283710,-123.124466,1984,37,honda,accord,False,False,False
2,excellent,rwd,diesel,vancouver,110000,white,67000,owner,clean,automatic,...,unknown,49.210453,-123.108330,2008,13,gmc,c7500,False,False,False
3,unknown,unknown,gas,vancouver,243000,unknown,4000,owner,clean,automatic,...,unknown,49.232300,-123.087400,2003,18,acura,rsx,True,False,False
4,salvage,unknown,gas,vancouver,225000,black,1500,owner,salvage,automatic,...,unknown,49.220000,-123.071000,2005,16,acura,rsx,False,False,False


In [82]:
cars.model.value_counts()

unknown               658
civic                 566
corolla               382
grand caravan         376
3                     339
                     ... 
allure cx               1
nissan versa note       1
f250 lariat             1
accent se sedan 4d      1
macra                   1
Name: model, Length: 6293, dtype: int64

In [81]:
#some cars have a year in the make column
years = [str(y) for y in range(1900, 2020)]
cars[cars.make_.isin(years)]['model']

95                 honda cr-v
146             mercedes c350
152      nissan pathfinder se
160                 gmc yukon
161                    bmw m6
                 ...         
37172      pontiac sunfire sl
37173       chevrolet equinox
37174    nissan pathfinder le
37233                ram 1500
37282                    jeep
Name: model, Length: 1649, dtype: object

In [62]:
#chevy -> chevrolet
#many of the cars makes are a year. If the year is the make, replace by the first word of the model
cars[cars.make_=='2019']

,condition,drive,fuel,location,odometer,paint color,price,sale type,title status,transmission,...,size,latitude,longitude,year,age,make_,model,pos_words,neg_words,low_text
297,like new,4wd,gas,"vancouver, bc",14000,black,73000,owner,clean,automatic,...,full-size,49.223740,-123.066870,2019,2,2019,ford expedition lmt,False,False,False
339,like new,fwd,electric,vancouver,27000,blue,49000,owner,clean,automatic,...,unknown,49.234400,-123.145100,2019,2,2019,kia niro ev sx touring,False,False,False
1347,like new,4wd,gas,vancouver,18000,white,25000,owner,clean,manual,...,unknown,49.249200,-123.110400,2019,2,2019,subaru impreza sport model,False,False,False
2051,like new,fwd,gas,west vancouver,27000,blue,13500,owner,clean,automatic,...,full-size,49.323760,-123.131490,2018,3,2019,rio sedan,False,False,False
2379,new,fwd,gas,burnaby bc,16200,white,12250,dealer,rebuilt,automatic,...,unknown,49.216840,-122.968600,2019,2,2019,kia rio 5,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36466,like new,4wd,gas,langley,21392,orange,26989,dealer,clean,automatic,...,mid-size,49.176743,-122.669044,2019,2,2019,jeep renegade,False,False,False
36557,like new,fwd,gas,langley,18527,grey,22989,dealer,clean,automatic,...,full-size,49.176710,-122.669044,2019,2,2019,chevrolet impala,False,False,False
36559,like new,fwd,gas,langley,22200,red,20989,dealer,clean,automatic,...,full-size,49.176598,-122.668830,2019,2,2019,chevrolet malibu,False,False,False
36561,like new,fwd,gas,langley,37785,blue,19989,dealer,clean,automatic,...,mid-size,49.176480,-122.668850,2019,2,2019,honda civic,False,False,False
